<html>
    <summary></summary>
         <div> <p></p> </div>
         <div style="font-size: 20px; width: 800px;"> 
              <h1>
               <left>Process MUSTANG Structural Alignments for Dihedral Angle Diffusion Model</left>
              </h1>
              <p><left>============================================================================</left> </p>
<pre>May, 2025
Dihedral Angle Diffusion (DAD) model for structural phylogenetics
Clementine Yan, Walter Xie, Alex Popinga, Alexei Drummond
Notebook by: Alex Popinga
</pre>
         </div>
    </p>

</html>

<details>
  <summary>Copyright info</summary>
<details>



#### This notebook will process the ferritin protein structural alignment by doing the following steps:
- ##### Step 1 - Extract the individual structures and the amino acid sequence alignment from the PDB structural alignment (Input: PDB alignment file; Output: PDB individual structure files, FASTA file).
    * ##### Step 1.a - Ensure structure meta-data (protein ID) is added to individual PDB files (MUSTANG removes in alignment file) and truncate both PDBs and FASTA alignment to ensure equal length of structures.
- ##### Step 2 - Compute alignment scores (RMSD, TM-score, Q-score).
- ##### Step 3 - Convert the Cartesian coordinates of alpha-Carbons in the individual PDB structures to dihedral angles ready for structural phylogenetic inference.

### STEP 1 - Define functions to extract the individual ferritin structures and amino acid sequences, then output as individual PDB files and FASTA files, respectively.

In [ ]:
import os
import re

def extract_ferritin_structures(input_pdb_file):
    with open(input_pdb_file, 'r') as file:
        lines = file.readlines()
    
    current_structure = []
    structure_id = None
    sequences = {}
    
    for line in lines:
        if line.startswith("HEADER"):  # Identify the start of a new structure
            if structure_id and current_structure:
                write_pdb(structure_id, current_structure)
                sequences[structure_id] = extract_sequence(current_structure)
            structure_id = line.split()[-1]  # Last word in HEADER is assumed to be the ID
            structure_id = re.sub(r"-tran\.pdb$", "", structure_id)  # Remove "-tran.pdb" suffix
            current_structure = [line]
        elif line.startswith("TER"):  # Termination of the current structure
            current_structure.append(line)
            if structure_id:
                write_pdb(structure_id, current_structure)
                sequences[structure_id] = extract_sequence(current_structure)
            current_structure = []
            structure_id = None
        elif structure_id:
            current_structure.append(line)
    
    write_fasta(input_pdb_file, sequences)

def write_pdb(structure_id, pdb_lines):
    filename = f"{structure_id}.pdb"
    with open(filename, 'w') as out_file:
        out_file.writelines(pdb_lines)
    print(f"Extracted: {filename}")

def extract_sequence(pdb_lines):
    amino_acids = {
        'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
        'GLN': 'Q', 'GLU': 'E', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
        'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
        'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
    }
    sequence = []
    seen_residues = set()
    
    for line in pdb_lines:
        if line.startswith("ATOM") and line[13:15].strip() == "CA":
            res_name = line[17:20].strip()
            res_num = line[22:26].strip()
            
            if (res_name, res_num) not in seen_residues:
                seen_residues.add((res_name, res_num))
                if res_name in amino_acids:
                    sequence.append(amino_acids[res_name])
    
    return "".join(sequence)

def write_fasta(input_pdb_file, sequences):
    fasta_filename = os.path.splitext(os.path.basename(input_pdb_file))[0] + "_sequences.fasta"
    with open(fasta_filename, 'w') as fasta_file:
        for structure_id, sequence in sequences.items():
            fasta_file.write(f">{structure_id}\n{sequence}\n")
    print(f"FASTA file created: {fasta_filename}")

#### Specify the alignment path and file name and call our extract_ferritin_structures function.

In [ ]:
if __name__ == "__main__":
    input_pdb_file_m = "MUSTANG Structural Alignments/ferritins.pdb"
    extract_ferritin_structures(input_pdb_file_m)

Extracted: 1BCFA.pdb.pdb
Extracted: 1jigA.pdb.pdb
Extracted: 1nfvA.pdb.pdb
Extracted: 1uvhA.pdb.pdb
Extracted: 2jd70.pdb.pdb
Extracted: 1o9rA.pdb.pdb
Extracted: 1vlgA.pdb.pdb
Extracted: 2uw1A.pdb.pdb
Extracted: 1dpsA.pdb.pdb
Extracted: 1jtsA.pdb.pdb
Extracted: 1otkA.pdb.pdb
Extracted: 1eumA.pdb.pdb
Extracted: 1krqA.pdb.pdb
Extracted: 1qghA.pdb.pdb
Extracted: 2chpA.pdb.pdb
Extracted: 2vzbA.pdb.pdb
Extracted: 1jgcA.pdb.pdb
Extracted: 1lb3A.pdb.pdb
Extracted: 1r03A.pdb.pdb
Extracted: 1ji4A.pdb.pdb
Extracted: 2fkzA.pdb.pdb
Extracted: 3e6sA.pdb.pdb
Extracted: 1ji5A.pdb.pdb
Extracted: 1tjoA.pdb.pdb
Extracted: 2fzfA.pdb.pdb
FASTA file created: ferritins_sequences.fasta


### Step 2 - Compute MUSTANG alignment scores (RMSD, TM-score, Q-score).

In [ ]:
import os
import numpy as np
from Bio import PDB
from itertools import combinations

class ProteinAlignmentEvaluator:
    def __init__(self, pdb_files):
        """Initialize with a list of PDB files containing aligned structures."""
        self.pdb_files = pdb_files
        self.parser = PDB.PDBParser(QUIET=True)
        
    def get_ca_coordinates(self, pdb_file):
        """Extracts C-alpha coordinates from a PDB file."""
        structure = self.parser.get_structure("protein", pdb_file)
        ca_coords = []
        for model in structure:
            for chain in model:
                for residue in chain:
                    if "CA" in residue:
                        ca_coords.append(residue["CA"].coord)
        return np.array(ca_coords)

    def compute_rmsd(self, coords1, coords2):
        """Computes RMSD between two aligned coordinate sets."""
        return np.sqrt(np.mean(np.sum((coords1 - coords2) ** 2, axis=1)))

    def compute_tm_score(self, coords1, coords2):
        """Computes TM-score based on aligned structures."""
        L = min(len(coords1), len(coords2))
        d0 = 1.24 * (L ** (1/3)) - 1.8  # TM-score normalization factor
        distances = np.sqrt(np.sum((coords1 - coords2) ** 2, axis=1))
        tm_score = np.sum(1 / (1 + (distances / d0) ** 2)) / L
        return tm_score

    def compute_q_score(self, coords1, coords2, threshold=4.0):
        """Computes Q-score: fraction of residue pairs within a distance threshold."""
        distances = np.sqrt(np.sum((coords1 - coords2) ** 2, axis=1))
        q_score = np.sum(distances < threshold) / len(distances)
        return q_score

    def evaluate(self):
        """Computes RMSD, TM-score, and Q-score for all pairs of aligned structures."""
        all_coords = {pdb: self.get_ca_coordinates(pdb) for pdb in self.pdb_files}

        results = []
        for pdb1, pdb2 in combinations(self.pdb_files, 2):
            coords1 = all_coords[pdb1]
            coords2 = all_coords[pdb2]
            
            if len(coords1) != len(coords2):
                print(f"Skipping {pdb1} and {pdb2} due to mismatched lengths.")
                continue

            rmsd = self.compute_rmsd(coords1, coords2)
            tm_score = self.compute_tm_score(coords1, coords2)
            q_score = self.compute_q_score(coords1, coords2)

            results.append((pdb1, pdb2, rmsd, tm_score, q_score))

        return results

# Example usage
if __name__ == "__main__":
    pdb_folder = "MUSTANG Structural Alignments/"

    pdb_files = sorted([os.path.join(pdb_folder, f) for f in os.listdir(pdb_folder) if f.endswith(".pdb")])

    evaluator = ProteinAlignmentEvaluator(pdb_files)
    results = evaluator.evaluate()

    # Print results
    print("PDB1\tPDB2\tRMSD\tTM-score\tQ-score")
    for r in results:
        print(f"{r[0]}\t{r[1]}\t{r[2]:.3f}\t{r[3]:.3f}\t{r[4]:.3f}")

### **STEP 3 - Convert protein structures into phi and psi dihedral angles**

In [ ]:
import csv
import math
from Bio import PDB

# Hydrophobicity scale (Kyte-Doolittle)
HYDROPHOBICITY_MAP = {
    "A": 1.8, "C": 2.5, "D": -3.5, "E": -3.5, "F": 2.8,
    "G": -0.4, "H": -3.2, "I": 4.5, "K": -3.9, "L": 3.8,
    "M": 1.9, "N": -3.5, "P": -1.6, "Q": -3.5, "R": -4.5,
    "S": -0.8, "T": -0.7, "V": 4.2, "W": -0.9, "Y": -1.3
}

# Three-letter to one-letter amino acid mapping
THREE_TO_ONE_MAP = {
    "ALA": "A", "CYS": "C", "ASP": "D", "GLU": "E", "PHE": "F",
    "GLY": "G", "HIS": "H", "ILE": "I", "LYS": "K", "LEU": "L",
    "MET": "M", "ASN": "N", "PRO": "P", "GLN": "Q", "ARG": "R",
    "SER": "S", "THR": "T", "VAL": "V", "TRP": "W", "TYR": "Y"
}

def compute_phi_psi(structure):
    # Extract phi and psi angles from a PDB structure 
    model = structure[0]  # Only use first model
    angles = []
    
    for chain in model:
        polypeptides = PDB.PPBuilder().build_peptides(chain)
        for poly_index, poly in enumerate(polypeptides):
            phi_psi_list = poly.get_phi_psi_list()
            residues = poly.get_sequence()
            for i, (phi, psi) in enumerate(phi_psi_list):
                residue = poly[i]
                resname = residue.resname
                one_letter_code = THREE_TO_ONE_MAP.get(resname, "?")
                hydrophobicity = HYDROPHOBICITY_MAP.get(one_letter_code, 0.0)
                
                phi_val = math.pi + phi if phi is not None else None
                psi_val = math.pi + psi if psi is not None else None
                
                angles.append([
                    chain.id, resname, residue.id[1],
                    phi_val, psi_val, hydrophobicity
                ])
    return angles

def write_csv(output_file, angles):
    # Write extracted angles to a CSV file
    with open(output_file, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Chain", "Residue", "ResidueNumber", "Phi", "Psi", "Hydrophobicity"])
        for row in angles:
            writer.writerow(row)

def process_pdb_file(input_pdb, output_csv):
    # Process PDB file to extract dihedral angles and write to CSV
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("protein", input_pdb)
    angles = compute_phi_psi(structure)
    write_csv(output_csv, angles)
    print(f"Processed {input_pdb} -> {output_csv}")

if __name__ == "__main__":
    input_pdb = "MUSTANG Structural Alignments/ferritins.pdb"  # Change this to your PDB file
    output_csv = "ferritin_dihedral_angles.csv"
    process_pdb_file(input_pdb, output_csv)


Processed MUSTANG Structural Alignments/ferritins.pdb -> ferritin_dihedral_angles.csv
